In [65]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup as bs
import wget
import re
from urllib.parse import unquote
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [66]:
#read csv master file
path_to_master_file = 'F:/Environmental Baseline Data/Version 4 - Final/Support files/list_of_Section_52_projects.csv'
master = pd.read_csv(path_to_master_file, encoding='ISO-8859-1')
print('shape of csv:', master.shape)
print('numer of unique project:', master['Application title'].nunique())
master.head()

shape of csv: (76, 6)
numer of unique project: 37


,Section,Application title,link to folder with ESA,Folder link,Company Name,Consultant Name
0,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,268693,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.
1,52,2004-10-07 - Application for the Construction ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,3892241,Imperial Oil Resources Ventures Limited (Macke...,Imperial Oil Resources Ventures Limited
2,52,2006-05-23 - Application for the Brunswick Pip...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,408713,Emera Brunswick Pipeline Company Ltd.,Jacques Whitford
3,52,"2007-03-09 - Application for Line 13 Transfer,...",https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,456315,Enbridge Southern Lights GP Inc.,TERA Environmental Consultants
4,52,2007-05-30 - Application for the Alberta Clipp...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,464743,Enbridge Pipelines Inc.,TERA Environmental Consultants


In [67]:
#convert folder links to list and create download string
urls_list = master['link to folder with ESA'].tolist()
urls_list_load = [str(x).replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/View/', 
                                 'https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/') for x in urls_list]
print('number of links:' , len(urls_list_load))

number of links: 76


In [68]:
#download all file links from each folder
all_esa = []

for x in urls_list_load:
    cookies = {'RDI-NumberOfRecords': '200'}
    r = requests.get(x, cookies = cookies)
    soup = bs(r.text, 'lxml')
    tds = soup.find_all('td', {'class' : 'nopadding break-anywhere'})
    file_name = [x.text for x in soup.find_all('a', {'class' : 'row-icon'})]
    hrefs = [a['href'] for a in soup.find_all('a', {'class' : 'row-icon'})]
    df = pd.DataFrame({'file_name' : file_name, 'hrefs' : hrefs, 'folder link' : str(x)})
    all_esa.append(df)

In [69]:
#select only files that are ESAs using regular expressions
df_esa = pd.concat(all_esa, axis = 0)
regex_esa = 'COVER|NV|EPP|EIS|ESA|nvironment|Horn|ocio|EIA|App 13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|A1X1|A1C3|A3S1|A3S2|Goldboro'
df_esa = df_esa[df_esa['file_name'].str.contains(regex_esa)] #only the rows that contain regex
df_esa['DataID'] = df_esa['hrefs'].str.replace('/REGDOCS/File/Download/', '')
df_esa['hrefs'] = df_esa['hrefs'].str.replace('/REGDOCS', 'https://apps.cer-rec.gc.ca/REGDOCS')
df_esa = df_esa.rename(columns={'hrefs' : 'download link'})

df_esa.head()

,file_name,download link,folder link,DataID
5,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268706
6,A0H8C1 - 13.0 EIA - Section 13.7 Wildlife Part 1,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268709
7,A0H8C2 - 13.0 EIA - Section 13.7 Wildlife Part 2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268712
8,A0H8C3 - 13.0 EIA - Section 13.8 to 13.13,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,269018
9,A0H8C4 - 13.1 App 13A - Alignment Sheets,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,269021


In [71]:
download_link = df_esa['download link'].tolist()
DataID = df_esa['DataID'].tolist()

In [72]:
save_folder = 'F:/Environmental Baseline Data/Version 4 - Final/PDF/'

for x, y in zip(download_link, DataID):
    download_url = 'http://docs2.cer-rec.gc.ca/ll-eng/llisapi.dll?func=ll&objId=' + y + '&objaction=download&viewType=1'
    r = requests.get(download_url)
    full_name = os.path.join(save_folder, (y +'.pdf')) 
    with open(full_name, 'wb') as file:
        file.write(r.content) 

1483221

4544963

4369127

2180117

3266671

...

9905765

15553163

10158939

15351865

14548815